In [1]:
import pandas as pd

from master_thesis.core.utils.reproducibility import seed_everything
from master_thesis.core.models.llama import load_model_and_tokenizer
from master_thesis.core.utils.prompts import load_prompt
from master_thesis.core.methods.conexp import CONEXPEstimator
from master_thesis.core.utils.reproducibility import save_results


DEVICE = "cuda"
PROMPT_TYPE = "few_shot"
POSITIVE_TOKEN = "true"
NEGATIVE_TOKEN = "false"
MODEL = "LLAMA_2_7B_CHAT"

DATA_DIR = "../../../../data"
DATASETS_DIR = f"{DATA_DIR}/datasets/base_experiments/car_vs_bike/test"
CACHE_DIR = f".cache/results"
CLASS_NAMES = ["bike", "car"]
LABEL_ASCPECT_NAMES = ["bike", "car"]
CONFOUNDING_ASPECT_NAMES = ["negative", "positive"]

In [2]:
seed_everything()

In [3]:
model, tokenizer = load_model_and_tokenizer(MODEL)

/home/mateusz_politycki/master-thesis-probing/.venv/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:732: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/mateusz_politycki/master-thesis-probing/.venv/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
TOKENS = [tokenizer.encode(NEGATIVE_TOKEN)[-1], tokenizer.encode(POSITIVE_TOKEN)[-1]]

### Prepare datasets and prompt

In [5]:
test = pd.read_csv(f"{DATASETS_DIR}/test.csv")
test_50_50 = pd.read_csv(f"{DATASETS_DIR}/test_50_50.csv")
test_95_5 = pd.read_csv(f"{DATASETS_DIR}/test_95_5.csv")

In [6]:
prompt = load_prompt(
    DATA_DIR,
    dataset_path="base_experiments/car_vs_bike",
    prompt_type=PROMPT_TYPE,
    prompt_aspect="label_aspect",
)

### Estimate CONEXP

In [7]:
conexp_etimator = CONEXPEstimator(model, tokenizer, prompt, TOKENS, DEVICE)

#### Test 50 50

In [8]:
test_50_50_results_label = conexp_etimator.evaluate(test_50_50, "label")

100%|██████████| 200/200 [01:24<00:00,  2.35it/s]


In [9]:
save_results(
    test_50_50_results_label,
    class_names=CLASS_NAMES,
    aspect_names=LABEL_ASCPECT_NAMES,
    save_path=f"{CACHE_DIR}/conexp/test_50_50/label_aspect.json",
)

In [10]:
test_50_50_results_label

{0: {0: 0.8947252532128275, 1: 0.8949891483183455},
 1: {0: 0.8947252532128275, 1: 0.8949891483183455}}

In [11]:
test_50_50_results_confounding = conexp_etimator.evaluate(test_50_50, "confounding")

100%|██████████| 200/200 [01:22<00:00,  2.43it/s]


In [12]:
save_results(
    test_50_50_results_confounding,
    class_names=CLASS_NAMES,
    aspect_names=CONFOUNDING_ASPECT_NAMES,
    save_path=f"{CACHE_DIR}/conexp/test_50_50/confounding_aspect.json",
)

In [13]:
test_50_50_results_confounding

{0: {0: 0.09424936545050933, 1: 0.09421040649474888},
 1: {0: 0.09424936545050933, 1: 0.09421040649474888}}

#### Test 95 5

In [14]:
test_95_5_results_label = conexp_etimator.evaluate(test_95_5, "label")

100%|██████████| 200/200 [01:22<00:00,  2.43it/s]


In [15]:
save_results(
    test_95_5_results_label,
    class_names=CLASS_NAMES,
    aspect_names=LABEL_ASCPECT_NAMES,
    save_path=f"{CACHE_DIR}/conexp/test_95_5/label_aspect.json",
)

In [16]:
test_95_5_results_label

{0: {0: 0.9805910286328262, 1: 0.9808196394790866},
 1: {0: 0.9805910286328262, 1: 0.9808196394790866}}

In [17]:
test_95_5_results_confounding = conexp_etimator.evaluate(test_95_5, "confounding")

100%|██████████| 200/200 [01:22<00:00,  2.43it/s]


In [18]:
save_results(
    test_95_5_results_confounding,
    class_names=CLASS_NAMES,
    aspect_names=CONFOUNDING_ASPECT_NAMES,
    save_path=f"{CACHE_DIR}/conexp/test_95_5/confounding_aspect.json",
)

In [19]:
test_95_5_results_confounding

{0: {0: 0.8807594169409876, 1: 0.8809526716373511},
 1: {0: 0.8807594169409876, 1: 0.8809526716373511}}